#  Anomaly detection in cellular networks

## Introduction

The purpose of this notebook is to solve a anomaly detection problem proposed as a competition in the Kaggle InClass platform.

## Problem description

### Context:

Traditionally, the design of a cellular network focuses on the optimization of energy and resources that guarantees a smooth operation even during peak hours (i.e. periods with higher traffic load). 
However, this implies that cells are most of the time overprovisioned of radio resources. 
Next generation cellular networks ask for a dynamic management and configuration in order to adapt to the varying user demands in the most efficient way with regards to energy savings and utilization of frequency resources. 
If the network operator were capable of anticipating to those variations in the users’ traffic demands, a more efficient management of the scarce (and expensive) network resources would be possible.
Current research in mobile networks looks upon Machine Learning (ML) techniques to help manage those resources. 
In this case, you will explore the possibilities of ML to detect abnormal behaviors in the utilization of the network that would motivate a change in the configuration of the base station.


### Objective

The objective of the network optimization team is to analyze traces of past activity, which will be used to train an ML system capable of classifying samples of current activity as:
 - 0 (normal): current activity corresponds to normal behavior of any working day and. Therefore, no re-configuration or redistribution of resources is needed.
 - 1 (unusual): current activity slightly differs from the behavior usually observed for that time of the day (e.g. due to a strike, demonstration, sports event, etc.), which should trigger a reconfiguration of the base station.

### Dataset

The dataset has been obtained from a real LTE deployment. During two weeks, different metrics were gathered from a set of 10 base stations, each having a different number of cells, every 15 minutes. 

The dataset is provided in the form of a csv file, where each row corresponds to a sample obtained from one particular cell at a certain time. Each data example contains the following features:

 - Time : hour of the day (in the format hh:mm) when the sample was generated.
 - CellName1: text string used to uniquely identify the cell that generated the current sample. CellName is in the form xαLTE, where x identifies the base station, and α the cell within that base station (see the example in the right figure).
 - PRBUsageUL and PRBUsageDL: level of resource utilization in that cell measured as the portion of Physical Radio Blocks (PRB) that were in use (%) in the previous 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - meanThrDL and meanThrUL: average carried traffic (in Mbps) during the past 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - maxThrDL and maxThrUL: maximum carried traffic (in Mbps) measured in the last 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - meanUEDL and meanUEUL: average number of user equipment (UE) devices that were simultaneously active during the last 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - maxUEDL and maxUEUL: maximum number of user equipment (UE) devices that were simultaneously active during the last 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - maxUE_UL+DL: maximum number of user equipment (UE) devices that were active simultaneously in the last 15 minutes, regardless of UL and DL.
 - Unusual: labels for supervised learning. A value of 0 determines that the sample corresponds to normal operation, a value of 1 identifies unusual behavior.

## Libraries

In [ ]:
import os
import sys
import random
random.seed(888) #set seed for reproducibility
from zipfile import ZipFile
from IPython.display import Image

#Analysis
import pyspark
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    print('WARN: Something wrong with pyspark library. Please check configuration settings!')
from pyspark.sql.types import DoubleType
    
#Feature Engineering
from pyspark.sql.functions import col, when, lit, array, explode, rand, udf
from pyspark.ml import PipelineModel
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
#Model Training
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#Model Registry
import pandas as pd
from sasctl import pzmm
from sasctl import Session
import getpass

    
# Reloads functions each time so you can edit a script and not need to restart the kernel
%load_ext autoreload
%autoreload 2

## Helpers

In [ ]:
def get_root_dir(src:str, max_nest:int) -> str:
    '''
    Specify paths and appending directories with relevant python source code.
    '''
    root_dir = os.curdir
    nest = 0
    while src not in os.listdir(root_dir) and nest < max_nest:
        root_dir = os.path.join(os.pardir, root_dir)     # Look up the directory structure for a src directory
        nest += 1
        
    # If you don't find the src directory, the root directory is this directory
    root_dir = os.path.abspath(root_dir) if nest < max_nest else os.path.abspath(
    os.curdir)
    
    return root_dir

def set_src(root_dir:str, src:str) -> str:
    '''
     Get the source directory and append path to access python packages/scripts within directory
    '''
    if src in os.listdir(root_dir):
        src_dir = os.path.join(root_dir, src)
        sys.path.append(src_dir)
    return sys.path[-1]

def set_folder(root_dir:str, folder:str) -> str:
    '''
    Set the folder path based on the folder name
    '''
    folder_path = os.path.join(
        root_dir, folder) if folder in os.listdir(root_dir) else os.curdir
    return folder_path

def set_path(path:str, dirname:str) -> str:
    '''
    '''
    return os.path.join(path, dirname)

def unzip(inpath:str, outpath:str) -> None:
    zf = ZipFile(inpath, 'r')
    zf.extractall(outpath)
    zf.close()
    
def metrics(dataframe, actual, predicted):
    '''
    Calculates evaluation metrics from predicted results
    
    Input:
    ---------
        dataframe: spark.sql.dataframe with the real and predicted values [object]
        actual:  Name of column with observed target values [string]
        predicted: Name of column with predicted values [string]
        
    
    Output:
    ---------
        None
    '''
       
    # Along each row are the actual values and down each column are the predicted
    dataframe = dataframe.withColumn(actual, col(actual).cast('integer'))
    dataframe = dataframe.withColumn(predicted, col(predicted).cast('integer'))
    cm = dataframe.crosstab(actual, predicted)
    cm = cm.sort(cm.columns[0], ascending = True)
    
    # Adds missing column in case just one class was predicted
    if not '0' in cm.columns:
        cm = cm.withColumn('0', lit(0))
    if not '1' in cm.columns:
        cm = cm.withColumn('1', lit(0))
    
    # Subsets values from confusion matrix
    zero = cm.filter(cm[cm.columns[0]] == 0.0)
    first_0 = zero.take(1)
    
    one = cm.filter(cm[cm.columns[0]] == 1.0)
    first_1 = one.take(1)
    
    tn = first_0[0][1]
    fp = first_0[0][2]
    fn = first_1[0][1]
    tp = first_1[0][2]
    
    # Calculate metrics from values in the confussion matrix
    if (tp == 0):
        acc = float((tp + tn) / (tp + tn + fp + fn))
        sen = 0
        spe = float((tn) / (tn + fp))
        prec = 0
        rec = 0
        f1 = 0
    elif (tn == 0):
        acc = float((tp + tn) / (tp + tn + fp + fn))
        sen = float((tp) / (tp + fn))
        spe = 0
        prec = float((tp) / (tp + fp))
        rec = float((tp) / (tp + fn))
        f1 = 2 * float((prec * rec) / (prec + rec))
    else:
        acc = float((tp + tn) / (tp + tn + fp + fn))
        sen = float((tp) / (tp + fn))
        spe = float((tn) / (tn + fp))
        prec = float((tp) / (tp + fp))
        rec = float((tp) / (tp + fn))
        f1 = 2 * float((prec * rec) / (prec + rec))

    # Print results
    print('Confusion Matrix and Statistics: \n')
    cm.show()
    
    print('True Positives:', tp)
    print('True Negatives:', tn)
    print('False Positives:', fp)
    print('False Negatives:', fn)
    print('Total:', dataframe.count(), '\n')
    
    print('Accuracy: {0:.2f}'.format(acc))
    print('Sensitivity: {0:.2f}'.format(sen))
    print('Specificity: {0:.2f}'.format(spe))
    print('Precision: {0:.2f}'.format(prec))
    print('Recall: {0:.2f}'.format(rec))
    print('F1-score: {0:.2f}'.format(f1))
    
    # Create spark dataframe with results
    l = [(acc, sen, spe, prec, rec, f1)]
    df = spark.createDataFrame(l, ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'Recall', 'F1'])

    return(df)

def get_output_variables(names, labels, eventprob):
    '''
    Given variable names, labels and event probability, 
    it creates dataframes for pzmm metadata generation
    :param names: 
    :param labels: 
    :param eventprob: 
    :return: outputVar
    '''
    outputVar = pd.DataFrame(columns=names)
    outputVar[names[0]] = [random.random(), random.random()]
    outputVar[names[1]] = [random.random(), random.random()]
    outputVar[names[2]] = labels
    outputVar[names[3]] = eventprob
    return outputVar

def zip_folder(folder_to_zip_path, rmtree=False):
    '''
    Given the folder to zip path,
    create an archive
    :param folder_to_zip_path: 
    :param rmtree: 
    :return: zipath
    '''
    path_sep = '/'
    root_dir = path_sep.join(folder_to_zip_path.split('/')[:-1])
    base_dir = folder_to_zip_path.split('/')[-1]
    zipath = shutil.make_archive(
        folder_to_zip_path,         # folder to zip
        'zip',                      # the archive format - or tar, bztar, gztar 
        root_dir=root_dir,          # folder to zip root
        base_dir=base_dir)          # folder to zip name
    if rmtree:
        shutil.rmtree(folder_to_zip_path) # remove .zip folder
    return zipath

extract1_udf = udf(lambda value: value[1].item(), DoubleType())

## Setup

In [ ]:
root_dir = get_root_dir('src', 5)
src_dir = set_src(root_dir, 'src')
data_dir = set_folder(root_dir, 'data')
raw_data_dir = set_path(data_dir, 'raw')
interim_data_dir = set_path(data_dir, 'interim')
processed_data_dir = set_path(data_dir, 'processed')
figures_dir = set_folder(root_dir, 'figures')
features_dir = set_folder(root_dir, 'features')
index_features_dir = set_path(features_dir, 'index')
ohe_features_dir = set_path(features_dir, 'ohe')
std_features_dir = set_path(features_dir, 'std')
models_dir = set_folder(root_dir, 'models')
deliverables_path = set_folder(root_dir, 'deliverables')
gbt_pipe_dir = set_path(deliverables_path, 'gbt_pipeline')

# 1. Data

## Initiate Spark session

In [ ]:
#If not exists create a spark session named Anomaly Detection where the master node is local
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("Anomaly Detection") \
    .getOrCreate()

In [ ]:
spark.getActiveSession()

## Load

### Set path

In [ ]:
train_path = set_path(processed_data_dir, 'ML-MATT-CompetitionQT1920_train_processed.parquet')
test_path = set_path(processed_data_dir, 'ML-MATT-CompetitionQT1920_test_processed.parquet')

### Load data

In [ ]:
train_df = spark.read.parquet(train_path)
test_df = spark.read.parquet(test_path)

In [ ]:
train_df.printSchema()

In [ ]:
train_df.show(5)

## 2. Load ML Pipeline

In [ ]:
gbt_pipe = PipelineModel.load(gbt_pipe_dir)

## 3. Test the pipe

In [ ]:
predictions_train = gbt_pipe.transform(train_df)
predictions_train.select('CellName', 'features', 'Unusual', 'rawPrediction', 'probability', 'prediction').show(5)
metrics(predictions_train, 'Unusual', 'prediction')

In [ ]:
predictions_test = gbt_pipe.transform(test_df)
predictions_test.select('CellName', 'features', 'Unusual', 'rawPrediction', 'probability', 'prediction').show(5)
metrics(predictions_test, 'Unusual', 'prediction')

## 4. Create Model for versioning in SAS Model Manager

As always, we create some metadata files to take advantages from SAS Model Manager.
In this case, because we want to migrate workload on GCP, we need as much info as possible. So we create:

1. requirement.txt
2. *.py with etl, train model and score data
3. *.json with all metainfo (inputs, outputs, model properties...)

### requirement.txt

In [ ]:
%%writefile ../deliverables/requirement.txt
numpy==1.19.4
pandas==1.1.4
pyspark==3.0.1
sasctl==1.5.4

### *.json with all metainfo

In [ ]:
# Base metainfo

inSample = train_df.toPandas()[:1]
js = pzmm.JSONFiles()

# inputVar.json
js.writeVarJSON(inSample, isInput=True, jPath=deliverables_path)

# outputVar.json
names=['P_UNUSUAL0', 'P_UNUSUAL1', 'EM_CLASSIFICATION', 'EM_EVENTPROBABILITY']
labels=['0', '1']
eventprob=0.5
outSample = get_output_variables(names, labels, eventprob)
js.writeVarJSON(outSample, isInput=False, jPath=deliverables_path)

# ModelProperties.json
modelname='pyspark_gbtClassifier'
target='Unusual'
predictors=['CellName', 'hour', 'minutes', 'PRBUsageUL', 'PRBUsageDL', 
            'meanThr_DL', 'meanThr_UL', 'maxThr_DL', 'maxThr_UL', 'meanUE_DL', 
            'meanUE_UL', 'maxUE_DL', 'maxUE_UL']

js.writeModelPropertiesJSON(modelName=modelname,
                                   modelDesc='A pyspark GBTClassifier for Network anomaly detection',
                                   targetVariable=target,
                                   modelType='Boosted Tree',
                                   modelPredictors=predictors,
                                   targetEvent=1,
                                   numTargetCategories=1,
                                   eventProbVar='EM_EVENTPROBABILITY',
                                   jPath=deliverables_path,
                                   modeler='ivnard')

In [ ]:
# Advanced metainfo: dmcas_fitstat.json, dmcas_roc.json, dmcas_lift.json

trainData = predictions_train.withColumn('p2', extract1_udf('probability')) \
                             .select('Unusual', 'p2').toPandas()
testData = predictions_test.withColumn('p2', extract1_udf('probability')) \
                             .select('Unusual', 'p2').toPandas()
js.calculateFitStat(trainData=trainData, testData=testData, jPath=deliverables_path)

print('Provide username and password to Viya Server login')
username = getpass.getpass()
password = getpass.getpass()
host = 'rusid1.rus.sas.com'
sess = Session(host, username, password, verify_ssl=False, protocol='http')
conn = sess.as_swat()
js.generateROCLiftStat(target, 1, conn, trainData=trainData, testData=testData, jPath=deliverables_path)

### *.py with etl, train model and score data

# Conclusion